Currently stacks using weighted average on predicted submissions files.

To do:
- Use folds in the initial layers and train a stacking a layer (boosted tree)

## Import data

In [6]:
import numpy as np
import pandas as pd

In [5]:
path = 'data/raw/train.csv'

full_path = os.path.join(dir_path, path)
df_train = pd.read_csv(full_path, header=0, index_col=0)
print("Dataset has {} rows, {} columns.".format(*df_train.shape))

Dataset has 95851 rows, 7 columns.


In [6]:
path = 'data/raw/test.csv'

full_path = os.path.join(dir_path, path)
df_test = pd.read_csv(full_path, header=0, index_col=0)
print("Dataset has {} rows, {} columns.".format(*df_test.shape))

Dataset has 226998 rows, 1 columns.


In [7]:
# fill NaN with string "unknown"
df_train.fillna('unknown',inplace=True)
df_test.fillna('unknown',inplace=True)

## Pre-processing

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
seed = 42
np.random.seed(seed)
test_size = 0.2
target = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
corpus = 'comment_text'

X = df_train[corpus]
y = df_train[target]


Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=test_size, random_state=seed)

## Load and predict using trained models

In [10]:
from sklearn.model_selection import ParameterGrid
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.pipeline import Pipeline
from keras.callbacks import CSVLogger
from sklearn.metrics import log_loss

Using TensorFlow backend.


In [ ]:
from keras.models import model_from_json

def load_model(model_name):
    # load json and create model
    json_file = open(model_name+'.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights(model_name+".h5")
    print("Loaded model from disk")
    return loaded_model

In [ ]:
# LSTM
model_name = ''
loaded_model = load_model(model_name)
y_pred_LSTM = loaded_model.predict(padded_test, verbose=1)

# LR
model_name = ''
loaded_model = load_model(model_name)
y_pred_LR = loaded_model.predict(padded_test, verbose=1)

## Stacking layer

In [4]:
import os

def load_data(path):
    full_path = os.path.join(os.path.realpath('..'), path)
    df = pd.read_csv(full_path, header=0, index_col=0)
    print("Dataset has {} rows, {} columns.".format(*df.shape))
    return df

In [9]:
LR = load_data('data/submissions/LR.csv')
LSTM = load_data('data/submissions/LSTM1.csv')

Dataset has 153164 rows, 6 columns.
Dataset has 153164 rows, 6 columns.


In [40]:
# Weighted averaging
score_lr = 0.188
score_lstm = 0.065
total = score_lr + score_lstm
df_stack = (1-score_lr/total)*LR + (1-score_lstm/total)*LSTM

In [41]:
df_stack.head()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
id,,,,,,
00001cee341fdb12,0.700484,0.143361,0.595480,0.026594,0.549922,0.346802
0000247867823ef7,0.021168,0.000261,0.008129,0.000247,0.017907,0.008166
00013b17ad220c46,0.016195,0.000409,0.001198,0.000103,0.001266,0.000253
00017563c3f7919a,0.001309,0.000315,0.001791,0.000108,0.000926,0.000172
00017695ad8997eb,0.030189,0.000776,0.003190,0.000229,0.000967,0.000272


## Submissions

In [47]:
model_name = 'LR_LSTM'
path = 'data/submissions/' + model_name + '.csv'
full_path = os.path.join(os.path.realpath('..'), path)
df_stack.to_csv(full_path, header=True, index=True)

## Misc

In [19]:
# Create stacking layer training set - LSTM predictions with true labels
stacking = pd.concat([predictions[0], predictions[1]])
for col in stacking.columns:
    stacking.rename(columns={col: col+'_LSTM'}, inplace=True)
stacking = stacking.join(X)
stacking = stacking.join(y)

# Evaluate 1st layer
for label in target:
    loss = log_loss(stacking[label], stacking[label+'_LSTM'])
    losses.append(loss)
    print("{} log loss is {} .".format(label, loss))
print("Combined log loss: {} .".format(np.mean(losses)))

# Save file
path = 'data/processed/stacking.csv'
full_path = os.path.join(dir_path, path)
ytest.to_csv(full_path, header=True, index=True)

toxic log loss is 0.11489777604597493 .
severe_toxic log loss is 0.023760985902965184 .
obscene log loss is 0.0592291762449135 .
threat log loss is 0.012569925774706866 .
insult log loss is 0.06995742146136677 .
identity_hate log loss is 0.02375914206328263 .
Combined log loss: 0.05122455304682492 .


In [20]:
# Use HOO to evaluate first layer combined log loss